In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras
board = keras.callbacks.TensorBoard(log_dir='./logs/artsnobs', histogram_freq=0, batch_size=100, write_graph=True)

#Dynamically change learning rate based on 'val_acc' over time
#change 0.8 to 0.75?
#factor = 0.75
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.625, patience=7, min_lr=0.001, verbose=1)

#Stop training when 'val_loss' stops improving after 15 epochs
#patience was 15
e_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

#Saves a checkpoint of the model every time its validation accuracy improves
chkpt = keras.callbacks.ModelCheckpoint("artsnobs_chkpt.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

#Next 4 lines allows MASSIVE images to be loaded in and used
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
PIL.Image.MAX_IMAGE_PIXELS = None

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
# model.add(Conv2D(32, (3, 3), input_shape=(175, 175, 3)))
model.add(Conv2D(32, (3, 3), input_shape=(500, 500, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
# model.add(BatchNormalization()) #added this
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3))) #changed 32 to 64
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3))) #added this
model.add(Activation('relu')) #added this
model.add(MaxPooling2D(pool_size=(2, 2))) #added this

In [3]:
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

# model.add(Dense(64)) #added this
# model.add(Activation('sigmoid')) #added this

model.add(Dropout(0.5))
model.add(Dense(3)) #3 for art_genre_set, 5 for dataset
model.add(Activation('softmax'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])
# model.summary()
model.load_weights('artsnobs_chkpt.hdf5') #BE AWARE IF THIS IS ENABLED

In [5]:
batch_size = 16 #change this from 16, used 100 for dataset
# train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=40, width_shift_range=0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest')

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/art_genre_set/train/', target_size = (500, 500), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
v_generator = test_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/art_genre_set/test/', target_size = (500, 500), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')

# train_generator = train_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/training_set/', target_size = (175, 175), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
# v_generator = test_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/validation_set/', target_size = (175,175), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
# # train_generator = train_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/training_set/', target_size = (150, 150), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')
# # v_generator = test_datagen.flow_from_directory('/Users/ethan/Documents/MLData/ArtSnobs/dataset/dataset_updated/validation_set/', target_size = (150,150), batch_size = batch_size, class_mode = 'categorical', color_mode='rgb')

Found 19011 images belonging to 3 classes.
Found 2112 images belonging to 3 classes.


In [ ]:
#change steps_per_epoch = 2500 to entire size of dataset (21123)
#change validation_steps = 1250 to half the entire size of the dataset (10561)

model.fit_generator(train_generator, steps_per_epoch = 21123 // batch_size, epochs = 250, validation_data =v_generator, validation_steps = 10561 // batch_size, callbacks = [board, e_stop, reduce_lr, chkpt], workers = 6, initial_epoch = 19)
model.save_weights('artsnobs_weights_correct_steps_per_epoch_valid_steps.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 20/250
1320/1320 [==============================] - 512s 388ms/step - loss: 0.8794 - acc: 0.5927 - val_loss: 0.9132 - val_acc: 0.5507

Epoch 00020: val_acc improved from -inf to 0.55066, saving model to artsnobs_chkpt.hdf5
Epoch 21/250
1320/1320 [==============================] - 492s 373ms/step - loss: 0.8664 - acc: 0.5988 - val_loss: 0.9250 - val_acc: 0.5639

Epoch 00021: val_acc improved from 0.55066 to 0.56392, saving model to artsnobs_chkpt.hdf5
Epoch 22/250
1320/1320 [==============================] - 494s 374ms/step - loss: 0.8673 - acc: 0.6015 - val_loss: 0.9258 - val_acc: 0.5545

Epoch 00022: val_acc did not improve from 0.56392
Epoch 23/250
1320/1320 [==============================] - 516s 391ms/step - loss: 0.8696 - acc: 0.5991 - val_loss: 0.9264 - val_acc: 0.5724

Epoch 00023: val_acc improved from 0.56392 to 0.57244, saving model to artsnobs_chkpt.hdf5
Epoch 24/250
1320/13

1320/1320 [==============================] - 501s 379ms/step - loss: 0.7833 - acc: 0.6481 - val_loss: 0.8970 - val_acc: 0.5777

Epoch 00056: ReduceLROnPlateau reducing learning rate to 0.152587890625.

Epoch 00056: val_acc did not improve from 0.60085
Epoch 57/250
1320/1320 [==============================] - 523s 396ms/step - loss: 0.7655 - acc: 0.6609 - val_loss: 0.9173 - val_acc: 0.5966

Epoch 00057: val_acc did not improve from 0.60085
Epoch 58/250
1320/1320 [==============================] - 492s 373ms/step - loss: 0.7769 - acc: 0.6558 - val_loss: 0.9051 - val_acc: 0.5966

Epoch 00058: val_acc did not improve from 0.60085
Epoch 59/250
1320/1320 [==============================] - 494s 375ms/step - loss: 0.7679 - acc: 0.6583 - val_loss: 0.8950 - val_acc: 0.5843

Epoch 00059: val_acc did not improve from 0.60085
Epoch 60/250
1320/1320 [==============================] - 485s 368ms/step - loss: 0.7700 - acc: 0.6548 - val_loss: 0.9401 - val_acc: 0.5848

Epoch 00060: val_acc did not impro

In [ ]:
#https://jovianlin.io/saving-loading-keras-models/
#use the link above to load the full model back into the notebook to continue training